In [1]:
import numpy as np
import networkx as nx
import random
import torch
from train import Train_Model
#from tqdm import tqdm, trange
from tqdm import tqdm_notebook as tqdm
import gc
from model import *
from utils import *

print(torch.__version__)
print(torch.cuda.is_available())

1.7.0
True


In [2]:
def pre_sample_for_ether(walk_times,adj_sparse, train_index, batch_size, save_name, do_walk):
    if do_walk:
        nodes_num = len(train_index)
        walks = torch.zeros(nodes_num, walk_times+1).long().cuda()
        batches = create_batches_forList(train_index, batch_size, True)
        i=0
        degrees = torch.tensor(adj_sparse.sum(1)).view(-1).cuda()
        candi_node = 0
        for batch in batches:
            walks[i*batch_size : i*batch_size + len(batch), 0] = torch.tensor(batch).cuda()
            candi_node = sparse_mx_to_torch_sparse_tensor(adj_sparse[batch.cpu().numpy()]).cuda()
            chosen_node = torch.zeros(len(batch), adj_sparse.shape[0]).cuda()
            for id in range(len(batch)):
                chosen_node[id][batch[id]] = 1.
            candi_node = ((- chosen_node + candi_node)> 0.0).float()
            for walk_id in range(walk_times):
                for x in range(candi_node.shape[0]):
                    if candi_node[x].sum()==0:
                        candi_node[x][batch[x]] = 1.

                p = candi_node
                new_node = torch.multinomial(p,1).squeeze(1)
                walks[i*batch_size : i*batch_size + len(batch), walk_id+1] = new_node
                for id in range(len(batch)):
                    chosen_node[id][new_node[id]] = 1.
                candi_node = candi_node - chosen_node + sparse_mx_to_torch_sparse_tensor(adj_sparse[new_node.cpu()]).cuda()
                candi_node = (candi_node> 0.0).float()
                
            i+=1
        torch.cuda.empty_cache()
        result1 = np.array(walks.cpu())
        io.savemat(save_name+'.mat',{save_name:result1})
        return walks
    else:
        walks = io.loadmat(save_name+'.mat')
        print("walks",walks)
        return torch.tensor(walks[save_name]).cuda()

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
sparse_adj, sparse_adj_train,sparse_adj_train_all,features, train_feature,train_feature_all,labels, train_labels, id_train, id_valid, id_test, num_labels = Origin_load_ether_data()
sparse_adj.setdiag(1.0) 
sparse_adj_train.setdiag(1.0) 
sparse_adj_train_all.setdiag(1.0) 

features = (1402220, 12) type = <class 'numpy.ndarray'>
train_index 572
val_index 82
test_index 162
adj torch.Size([1402220, 1402220])
features torch.Size([1402220, 12])


/home/ubuntu/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [4]:
for walk in range(1,10):
    for seed in range(10,20):
        torch.manual_seed(seed)
        random.seed(seed)
        torch.cuda.manual_seed(seed)
        np.random.seed(seed)
        save_name = 'seed='+str(seed)+'_walk='+str(walk)
        print('process...'+'train_'+save_name)
        pre_sample_for_ether(walk,sparse_adj_train_all, id_train, 32, './walks_ether_random/train_'+save_name, False)
        print('process...'+'valid_'+save_name)
        pre_sample_for_ether(walk,sparse_adj, id_valid, 32, './walks_ether_random/valid_'+save_name, False)
        print('process...'+'test_'+save_name)
        pre_sample_for_ether(walk,sparse_adj, id_test, 32, './walks_ether_random/test_'+save_name, False)

process...train_seed=10_walk=1
walks {'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Sun Dec 13 13:49:50 2020', '__version__': '1.0', '__globals__': [], './walks_ether_random/train_seed=10_walk=1': array([[     0,  62606],
       [     1, 143128],
       [     2, 443894],
       ...,
       [   569, 292136],
       [   570, 417310],
       [   571, 249294]])}
process...valid_seed=10_walk=1
walks {'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Sun Dec 13 13:49:50 2020', '__version__': '1.0', '__globals__': [], './walks_ether_random/valid_seed=10_walk=1': array([[     2, 231553],
       [     4, 251959],
       [    11, 544419],
       [    27, 134340],
       [    42,    180],
       [    53, 558546],
       [    71,  27411],
       [    75,    195],
       [    91, 376081],
       [    93, 601220],
       [   103, 398634],
       [   118, 302875],
       [   119, 631243],
       [   121, 630848],
       [   154, 241864],
       [   173, 549573],
    

process...test_seed=16_walk=4
walks {'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Sat Dec 12 16:15:32 2020', '__version__': '1.0', '__globals__': [], './walks_ether_random/test_seed=16_walk=4': array([[      0,  325757,  104620,  468432,  299930],
       [      5,  150160,   25641,  346809,     195],
       [     15,  290632,  254954,  351569,  229354],
       [     17,  553575,  597989,  369758,  564851],
       [     19,     514,   76129,  152225,  251028],
       [     25,  261193,  336735,  498862,  605395],
       [     28,  520283,     803,    4397,  101505],
       [     37,  481815,  585882,     158,  153746],
       [     41,  280732,  280732,  280732,  280732],
       [     48,  605975,     238,     623,      61],
       [     51,  515789,     654,     437,     417],
       [     52,     803,  316947,  535438,  564113],
       [     59,  381662,  632032,    6661,  307113],
       [     62,   19424,  158959,  531450,  172196],
       [     67,  323105,  306

FileNotFoundError: [Errno 2] No such file or directory: './walks_ether_random/test_seed=17_walk=6.mat'